## Libraries

In [1]:
!pip install lightgbm
!pip install category_encoders
!pip install citylearn==2.1b9


# Because of the Wrapper Error
!pip install keras==2.12.0
!pip install tensorflow==2.12.0

# Then you have to reinitiate the kernel! and install scikit-optimize!
!pip install scikit-optimize
!pip install sktime
!pip install skforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 24.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 50.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 41.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 29.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 33.5 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827620 sha256=21dbb20c45dffb00d0e4e50c0880e5a4e6e8210e909167a2d7f189357396178b
  Stored in directory: /home/philaupk/.cache/pip/wheels/b9/22/6d/3e7b32d98451b4cd9d12417052affbeeeea012955d437da1da

In [30]:
import os
import glob
import numpy as np 
import pandas as pd 
import lightgbm as lgb
import category_encoders as ce
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor

import matplotlib.ticker as ticker
import re
from sklearn.model_selection import GridSearchCV
from joblib import dump, load
import joblib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import AdaBoostRegressor
import time
from tqdm.auto import tqdm
import json

from citylearn.citylearn import CityLearnEnv
from my_models.user_model import SubmissionModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
import logging
from sklearn.ensemble import VotingRegressor

logging.getLogger('tensorflow').setLevel(logging.ERROR)
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.base import BaseEstimator, RegressorMixin
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from keras.layers import Layer
import tensorflow as tf
import keras.backend as K
from tensorflow.keras.utils import register_keras_serializable

import numpy as np
from sklearn.model_selection import train_test_split
from skopt import gp_minimize
from functools import partial
from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, Dense
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import StandardScaler

import skforecast as skforecast
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)


## Simulator 

In [31]:
# Create a test env
class WrapperEnv:
    """
    Env to wrap provide Citylearn Env data without providing full env
    Preventing attribute access outside of the available functions
    """
    def __init__(self, env_data):
        self.observation_names = env_data['observation_names']
        self.action_names = env_data['action_names']
        self.observation_space = env_data['observation_space']
        self.action_space = env_data['action_space']
        self.time_steps = env_data['time_steps']
        self.seconds_per_time_step = env_data['seconds_per_time_step']
        self.random_seed = env_data['random_seed']
        self.buildings_metadata = env_data['buildings_metadata']
        self.episode_tracker = env_data['episode_tracker']
    
    def get_metadata(self):
        return {'buildings': self.buildings_metadata}

In [32]:
def create_citylearn_env(config):
    env = CityLearnEnv(config.SCHEMA)

    env_data = dict(
        observation_names = env.observation_names,
        action_names = env.action_names,
        observation_space = env.observation_space,
        action_space = env.action_space,
        time_steps = 35039,
        buildings_metadata = env.get_metadata()['buildings'],
        num_buildings = len(env.buildings),
        building_names = [b.name for b in env.buildings],
        b0_pv_capacity = env.buildings[0].pv.nominal_power,
    )

    # Turn off actions for all buildings and do not simulate power outage (forecasting only).
    for b in env.buildings:
        b.ignore_dynamics = True
        b.simulate_power_outage = False

    return env, env_data

In [33]:
class Config:
    data_dir = './data/'
    SCHEMA = os.path.join(data_dir, 'schemas/warm_up/schema.json')
config = Config()

In [34]:
env, env_data = create_citylearn_env(config)

## Generation of the Features

In [35]:
# Dataframes

b_1_dataframe = pd.DataFrame(columns=['day_type', 'hour', 'outdoor_dry_bulb_temperature', 'outdoor_dry_bulb_temperature_predicted_6h', 
                                      'outdoor_dry_bulb_temperature_predicted_12h', 'outdoor_dry_bulb_temperature_predicted_24h', 'diffuse_solar_irradiance',
                                      'diffuse_solar_irradiance_predicted_6h', 'diffuse_solar_irradiance_predicted_12h', 
                                      'diffuse_solar_irradiance_predicted_24h', 'direct_solar_irradiance', 'direct_solar_irradiance_predicted_6h',
                                      'direct_solar_irradiance_predicted_12h', 'direct_solar_irradiance_predicted_24h', 'carbon_intensity', 
                                      'indoor_dry_bulb_temperature', 'non_shiftable_load', 'solar_generation', 'dhw_storage_soc', 'electrical_storage_soc', 
                                      'electricity_pricing', 'electricity_pricing_predicted_6h', 
                                      'electricity_pricing_predicted_12h', 'electricity_pricing_predicted_24h', 'cooling_demand',
                                      'dhw_demand','indoor_dry_bulb_temperature_set_point','occupant_count','net_electricity_consumption'])

b_2_dataframe = pd.DataFrame(columns=['day_type', 'hour', 'outdoor_dry_bulb_temperature', 'outdoor_dry_bulb_temperature_predicted_6h', 
                                      'outdoor_dry_bulb_temperature_predicted_12h', 'outdoor_dry_bulb_temperature_predicted_24h', 'diffuse_solar_irradiance',
                                      'diffuse_solar_irradiance_predicted_6h', 'diffuse_solar_irradiance_predicted_12h', 
                                      'diffuse_solar_irradiance_predicted_24h', 'direct_solar_irradiance', 'direct_solar_irradiance_predicted_6h',
                                      'direct_solar_irradiance_predicted_12h', 'direct_solar_irradiance_predicted_24h', 'carbon_intensity', 
                                      'indoor_dry_bulb_temperature', 'non_shiftable_load', 'solar_generation', 'dhw_storage_soc', 'electrical_storage_soc', 
                                      'electricity_pricing', 'electricity_pricing_predicted_6h', 
                                      'electricity_pricing_predicted_12h', 'electricity_pricing_predicted_24h', 'cooling_demand',
                                      'dhw_demand','indoor_dry_bulb_temperature_set_point','occupant_count','net_electricity_consumption'])

b_3_dataframe = pd.DataFrame(columns=['day_type', 'hour', 'outdoor_dry_bulb_temperature', 'outdoor_dry_bulb_temperature_predicted_6h', 
                                      'outdoor_dry_bulb_temperature_predicted_12h', 'outdoor_dry_bulb_temperature_predicted_24h', 'diffuse_solar_irradiance',
                                      'diffuse_solar_irradiance_predicted_6h', 'diffuse_solar_irradiance_predicted_12h', 
                                      'diffuse_solar_irradiance_predicted_24h', 'direct_solar_irradiance', 'direct_solar_irradiance_predicted_6h',
                                      'direct_solar_irradiance_predicted_12h', 'direct_solar_irradiance_predicted_24h', 'carbon_intensity', 
                                      'indoor_dry_bulb_temperature', 'non_shiftable_load', 'solar_generation', 'dhw_storage_soc', 'electrical_storage_soc', 
                                      'electricity_pricing', 'electricity_pricing_predicted_6h', 
                                      'electricity_pricing_predicted_12h', 'electricity_pricing_predicted_24h', 'cooling_demand',
                                      'dhw_demand','indoor_dry_bulb_temperature_set_point','occupant_count','net_electricity_consumption'])

b_dataframe_list = [b_1_dataframe,b_2_dataframe,b_3_dataframe]


In [36]:
for idx, b in enumerate(env.buildings):
    
    con = b.net_electricity_consumption
    print(str(len(con)))
    
env.buildings[0].net_electricity_consumption

1
1
1


array([0.67788136], dtype=float32)

In [37]:
# Generate the Datasets for the different buildings:
# Here I only need to simulate the ones, which are not present in the dataset:

for idx, b in enumerate(env.buildings):
    indoor_dry_bulb_temperature           = b.energy_simulation.indoor_dry_bulb_temperature
    non_shiftable_load                    = b.energy_simulation.non_shiftable_load
    solar_generation                      = b.energy_simulation.solar_generation
    dhw_storage_soc                       = b.dhw_storage.soc
    electrical_storage_soc                = b.electrical_storage.soc
    cooling_demand                        = b.energy_simulation.cooling_demand
    dhw_demand                            = b.energy_simulation.dhw_demand
    indoor_dry_bulb_temperature_set_point = b.energy_simulation.indoor_dry_bulb_temperature_set_point
    occupant_count                        = b.occupant_count.repeat(720)
    net_electricity_consumption           = b.net_electricity_consumption.repeat(720)
    
    # After the generation of the different features I will add the global features (which are independend from the houses!)
    day_type         = env.buildings[0].energy_simulation.day_type
    hour             = env.buildings[0].energy_simulation.hour
    carbon_intensity = env.buildings[0].carbon_intensity.carbon_intensity

    # Loading the local features
    filepath = 'data/schemas/warm_up/'

    pricing    = pd.read_csv(filepath + 'pricing.csv')
    weather    = pd.read_csv(filepath + 'weather.csv')

    electricity_pricing                = pricing['Electricity Pricing [$/kWh]']
    electricity_pricing_predicted_6h   = pricing['6h Prediction Electricity Pricing [$/kWh]']
    electricity_pricing_predicted_12h  = pricing['12h Prediction Electricity Pricing [$/kWh]']
    electricity_pricing_predicted_24h  = pricing['24h Prediction Electricity Pricing [$/kWh]']

    outdoor_dry_bulb_temperature                = weather['Outdoor Drybulb Temperature (C)']
    outdoor_dry_bulb_temperature_predicted_6h   = weather['6h Outdoor Drybulb Temperature (C)']
    outdoor_dry_bulb_temperature_predicted_12h  = weather['12h Outdoor Drybulb Temperature (C)']
    outdoor_dry_bulb_temperature_predicted_24h  = weather['24h Outdoor Drybulb Temperature (C)']

    diffuse_solar_irradiance                    = weather['Diffuse Solar Radiation (W/m2)']
    diffuse_solar_irradiance_predicted_6h       = weather['6h Diffuse Solar Radiation (W/m2)']
    diffuse_solar_irradiance_predicted_12h      = weather['12h Diffuse Solar Radiation (W/m2)']
    diffuse_solar_irradiance_predicted_24h      = weather['24h Diffuse Solar Radiation (W/m2)']

    direct_solar_irradiance                     = weather['Direct Solar Radiation (W/m2)']
    direct_solar_irradiance_predicted_6h        = weather['6h Direct Solar Radiation (W/m2)']
    direct_solar_irradiance_predicted_12h       = weather['12h Direct Solar Radiation (W/m2)']
    direct_solar_irradiance_predicted_24h       = weather['24h Direct Solar Radiation (W/m2)']
    
    # Generate the Dataframe for the training
    b_dataframe_list[idx]['day_type']                                   = day_type
    b_dataframe_list[idx]['hour']                                       = hour
    b_dataframe_list[idx]['outdoor_dry_bulb_temperature']               = outdoor_dry_bulb_temperature
    b_dataframe_list[idx]['outdoor_dry_bulb_temperature_predicted_6h']  = outdoor_dry_bulb_temperature_predicted_6h
    b_dataframe_list[idx]['outdoor_dry_bulb_temperature_predicted_12h'] = outdoor_dry_bulb_temperature_predicted_12h
    b_dataframe_list[idx]['outdoor_dry_bulb_temperature_predicted_24h'] = outdoor_dry_bulb_temperature_predicted_24h
    b_dataframe_list[idx]['diffuse_solar_irradiance']                   = diffuse_solar_irradiance
    b_dataframe_list[idx]['diffuse_solar_irradiance_predicted_6h']      = diffuse_solar_irradiance_predicted_6h
    b_dataframe_list[idx]['diffuse_solar_irradiance_predicted_12h']     = diffuse_solar_irradiance_predicted_12h
    b_dataframe_list[idx]['diffuse_solar_irradiance_predicted_24h']     = diffuse_solar_irradiance_predicted_24h
    b_dataframe_list[idx]['direct_solar_irradiance']                    = direct_solar_irradiance
    b_dataframe_list[idx]['direct_solar_irradiance_predicted_6h']       = direct_solar_irradiance_predicted_6h
    b_dataframe_list[idx]['direct_solar_irradiance_predicted_12h']      = direct_solar_irradiance_predicted_12h
    b_dataframe_list[idx]['direct_solar_irradiance_predicted_24h']      = direct_solar_irradiance_predicted_24h
    b_dataframe_list[idx]['carbon_intensity']                           = carbon_intensity
    b_dataframe_list[idx]['indoor_dry_bulb_temperature']                = indoor_dry_bulb_temperature
    b_dataframe_list[idx]['non_shiftable_load']                         = non_shiftable_load
    b_dataframe_list[idx]['solar_generation']                           = solar_generation
    b_dataframe_list[idx]['dhw_storage_soc']                            = dhw_storage_soc
    b_dataframe_list[idx]['electrical_storage_soc']                     = electrical_storage_soc
    b_dataframe_list[idx]['electricity_pricing']                        = electricity_pricing
    b_dataframe_list[idx]['electricity_pricing_predicted_6h']           = electricity_pricing_predicted_6h
    b_dataframe_list[idx]['electricity_pricing_predicted_12h']          = electricity_pricing_predicted_12h
    b_dataframe_list[idx]['electricity_pricing_predicted_24h']          = electricity_pricing_predicted_24h
    b_dataframe_list[idx]['cooling_demand']                             = cooling_demand
    b_dataframe_list[idx]['dhw_demand']                                 = dhw_demand
    b_dataframe_list[idx]['indoor_dry_bulb_temperature_set_point']      = indoor_dry_bulb_temperature_set_point
    b_dataframe_list[idx]['occupant_count']                             = occupant_count
    b_dataframe_list[idx]['net_electricity_consumption']                = net_electricity_consumption

### Feature Selection

In [ ]:
# Save the important features into files
b = 1
for data in b_dataframe_list:
    feature_selection(data,'cooling_demand')
    feature_selection(data,'dhw_demand')
    feature_selection(data,'non_shiftable_load')
    feature_selection(data,'carbon_intensity')
    feature_selection(data,'solar_generation')
    b = b + 1

In [ ]:
def feature_selection(data,obs_feature):
    # Split the dataset into features and target
    X = data
    y = data[obs_feature]
    
    # Apply Information Gain
    ig = mutual_info_regression(X, y)

    # Create a dictionary of feature importance scores
    feature_scores = {}
    i = 0
    for (columnName, columnData) in data.items():
        feature_scores[columnName] = ig[i]
        i = i + 1
    # Sort the features by importance score in descending order
    sorted_features = sorted(feature_scores.items(), key=lambda x: x[1], reverse=True)

    f_l = []
    s_l = []
    a_l = []
    a_l_s = []
    # Print the feature importance scores and the sorted features
    for feature, score in sorted_features:
        a_l.append(feature)
        a_l_s.append(score)
        if score > 0.10:
            # save the features
            f_l.append(feature)
            s_l.append(score)
            
    dic = {'feature': f_l, 'score': s_l}
    dic_a = {'feature': a_l, 'score': a_l_s}
    df2 = pd.DataFrame(dic_a)
    df = pd.DataFrame(dic)
    df.to_csv('data/features/feature_importance_'+str(obs_feature)+'.csv')
    

## Train the Predictors (LightGBM)

In [ ]:
model_type = 'lgb'
hyperparameter = False

if 'BahdanauAttention' not in tf.keras.utils.get_custom_objects():
    register_keras_serializable('BahdanauAttention')(BahdanauAttention)

## Building Level Predictors

### 1.) Cooling Load (kWh)

In [66]:

i = 1
for b in b_dataframe_list:
    if model_type == 'xgb':
        xgb = XGBoost_Model(b,hyperparameter,'cooling_demand')
        joblib.dump(xgb, 'my_models/models/cooling_demand_model_b'+str(i)+'_xgb.pkl')
    if model_type == 'lgb':
        print("---------_-------------")
        lgb_ = LightGBM_Model(b,hyperparameter,'cooling_demand')
        joblib.dump(lgb_,'my_models/models/LightGBM/cooling_demand_model_b'+str(i)+'_wo.h5')
    if model_type == 'fusion':
        lgb_model  = LightGBM_Model(b,hyperparameter,'cooling_demand')
        lstm_model = LSTM_Model(b,hyperparameter,'cooling_demand')
        
        joblib.dump(lgb_model, 'my_models/models/fusion/LightGBM/cooling_demand_model_b'+str(i)+'.pkl')
        joblib.dump(lstm_model, 'my_models/models/fusion/LSTM/cooling_demand_model_b'+str(i)+'.pkl')

    if model_type == 'lstm':
        lstm_model = LSTM_Bi_Model(b,hyperparameter,'cooling_demand')
        lstm_model.save('my_models/models/LSTM_BiAttention/cooling_demand_model_b'+str(i)+'_hyper.h5')
        
    if model_type == 'bi-lstm':
        lstm_model = LSTM_Bi_Model_hyper(b,hyperparameter,'cooling_demand')
        lstm_model.save('my_models/models/LSTM_Bi_Model/cooling_demand_model_b'+str(i)+'_hyper.h5')
    i = i + 1

---------_-------------
29
---------_-------------
29
---------_-------------
29


### 2.) DHW Load (kWh)

In [67]:
i = 1

for b in b_dataframe_list:

    if model_type == 'xgb':
        xgb = XGBoost_Model(b,hyperparameter,'dhw_demand')
        joblib.dump(xgb, 'my_models/models/dhw_demand_model_b'+str(i)+'_xgb.pkl')
    if model_type == 'lgb':
        print("---------_-------------")
        lgb_ = LightGBM_Model(b,hyperparameter,'dhw_demand')
        joblib.dump(lgb_, 'my_models/models/LightGBM/dhw_demand_model_b'+str(i)+'_wo.h5')
    if model_type == 'fusion':
        lgb_model  = LightGBM_Model(b,hyperparameter,'dhw_demand')
        lstm_model = LSTM_Model(b,hyperparameter,'dhw_demand')
        
        joblib.dump(lgb_model, 'my_models/models/fusion/LightGBM/dhw_demand_model_b'+str(i)+'.pkl')
        joblib.dump(lstm_model, 'my_models/models/fusion/LSTM/dhw_demand_model_b'+str(i)+'.pkl')
        
    if model_type == 'lstm':
        lstm_model = LSTM_Bi_Model(b,hyperparameter,'dhw_demand')
        lstm_model.save('my_models/models/LSTM_BiAttention/dhw_demand_model_b'+str(i)+'_hyper.h5')
                        
    if model_type == 'bi-lstm':
        lstm_model = LSTM_Bi_Model_hyper(b,hyperparameter,'dhw_demand')
        lstm_model.save('my_models/models/LSTM_Bi_Model/dhw_demand_model_b'+str(i)+'_hyper.h5')

    i = i + 1

---------_-------------
29
---------_-------------
29
---------_-------------
29


### 3.) Equipment Electric Power (kWh)

In [68]:
i = 1
for b in b_dataframe_list:
    
    if model_type == 'xgb':
        xgb = XGBoost_Model(b,hyperparameter,'non_shiftable_load')
        joblib.dump(xgb, 'my_models/models/Equipment_Electric_Power_model_b'+str(i)+'_new_xgb.pkl')
    if model_type == 'lgb':
        lgb_ = LightGBM_Model(b,hyperparameter,'non_shiftable_load')
        joblib.dump(lgb_, 'my_models/models/LightGBM/Equipment_Electric_Power_model_b'+str(i)+'_wo.h5')
    if model_type == 'fusion':
        lgb_model  = LightGBM_Model(b,hyperparameter,'non_shiftable_load')
        
        joblib.dump(lgb_model, 'my_models/models/fusion/LightGBM/Equipment_Electric_Power_model_b'+str(i)+'.pkl')
        joblib.dump(lstm_model, 'my_models/models/fusion/LSTM/Equipment_Electric_Power_model_b'+str(i)+'.pkl')
        
    if model_type == 'lstm':
        lstm_model = LSTM_Bi_Model(b,hyperparameter,'non_shiftable_load')
        lstm_model.save('my_models/models/LSTM_BiAttention/Equipment_Electric_Power_model_b'+str(i)+'_hyper.h5')

    if model_type == 'bi-lstm':
        lstm_model = LSTM_Bi_Model_hyper(b,hyperparameter,'non_shiftable_load')
        lstm_model.save('my_models/models/LSTM_Bi_Model/Equipment_Electric_Power_model_b'+str(i)+'_hyper.h5')
    i = i + 1

29
29
29


# Neighbourhood Level Predictors

### 1.) Carbon Intensity (kgCO2e/kWh)

In [80]:
# combine the datasets to one since we only have one CI 
comb = pd.concat(b_dataframe_list, ignore_index=True)
# Reset the index without adding an additional index column
comb.reset_index(drop=True, inplace=True)
    
if model_type == 'xgb':
    xgb = XGBoost_Model(comb,hyperparameter,'carbon_intensity')
    joblib.dump(xgb, 'my_models/models/Carbon_Intensity_Power_model_xgb.pkl')
if model_type == 'lgb':
    lgb_ = LightGBM_Model(comb,hyperparameter,'carbon_intensity')
    joblib.dump(lgb_, 'my_models/models/LightGBM/Carbon_Intensity_Power_model_wo.h5')
if model_type == 'fusion':
    lgb_model  = LightGBM_Model(comb,hyperparameter,'carbon_intensity')
    lstm_model = LSTM_Model(comb,hyperparameter,'carbon_intensity')
        
    joblib.dump(lgb_model, 'my_models/models/fusion/LightGBM/Carbon_Intensity_model.pkl')
    joblib.dump(lstm_model, 'my_models/models/fusion/LSTM/Carbon_Intensity_model.pkl')

if model_type == 'lstm':
    lstm_model = LSTM_Bi_Model(comb,hyperparameter,'carbon_intensity')
    lstm_model.save('my_models/models/LSTM_BiAttention/Carbon_Intensity_model_hyper.h5')

if model_type == 'bi-lstm':
    lstm_model = LSTM_Bi_Model_hyper(comb,hyperparameter,'carbon_intensity')
    lstm_model.save('my_models/models/LSTM_Bi_Model/Carbon_Intensity_model_hyper.h5')


29


### 2.) Solar Genetation (ConCat)

In [83]:
# combine the datasets to one since we only have one CI 
comb = pd.concat(b_dataframe_list, ignore_index=True)
# Reset the index without adding an additional index column
comb.reset_index(drop=True, inplace=True)

if model_type == 'xgb':
    xgb = XGBoost_Model(comb,hyperparameter,'solar_generation')
    joblib.dump(xgb, 'my_models/models/solar_generation_Power_model_xgb.pkl')
if model_type == 'lgb':
    lgb_ = LightGBM_Model(comb,hyperparameter,'solar_generation')
    joblib.dump(lgb_, 'my_models/models/LightGBM/solar_generation_model_wo.h5')
if model_type == 'fusion':
    lgb_model  = LightGBM_Model(comb,hyperparameter,'solar_generation')
    lstm_model = LSTM_Model(comb,hyperparameter,'carbon_intensity')
        
    joblib.dump(lgb_model, 'my_models/models/fusion/LightGBM/solar_generation_model.pkl')
    joblib.dump(lstm_model, 'my_models/models/fusion/LSTM/solar_generation_model_scaled.pkl')

if model_type == 'lstm':
    lstm_model = LSTM_Bi_Model(comb,hyperparameter,'solar_generation')
    lstm_model.save('my_models/models/LSTM_BiAttention/solar_generation_model_hyper.h5')

if model_type == 'bi-lstm':
    lstm_model = LSTM_Bi_Model_hyper(comb,hyperparameter,'solar_generation')
    lstm_model.save('my_models/models/LSTM_Bi_Model/solar_generation_model_hyper.h5')

0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
          ...   
2155    2.211529
2156    0.000000
2157    0.000000
2158    0.000000
2159    0.000000
Name: solar_generation, Length: 2160, dtype: float32
29


In [79]:
comb['solar_generation']

0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
          ...   
2155    2.211529
2156    0.000000
2157    0.000000
2158    0.000000
2159    0.000000
Name: solar_generation, Length: 2160, dtype: float32

### 2.) Solar Generation (W/kW)

In [ ]:
i = 1
for b in b_dataframe_list:
    
    if model_type == 'xgb':
        xgb = XGBoost_Model(b,hyperparameter,'solar_generation')
        joblib.dump(xgb, 'my_models/models/solar_generation_model_b'+str(i)+'_xgb.pkl')
    if model_type == 'lgb':
        lgb = LightGBM_Model(b,hyperparameter,'solar_generation')
        joblib.dump(lgb, 'my_models/models/solar_generation_model_b'+str(i)+'_lightgbm.pkl')
    if model_type == 'fusion':
        lgb_model  = LightGBM_Model(b,hyperparameter,'solar_generation')
        lstm_model = LSTM_Model(b,hyperparameter,'solar_generation')
        
        joblib.dump(lgb_model, 'my_models/models/fusion/LightGBM/solar_generation_model_b'+str(i)+'.pkl')
        joblib.dump(lstm_model, 'my_models/models/fusion/LSTM/solar_generation_model_b'+str(i)+'.pkl')
    if model_type == 'lstm':
        lstm_model = LSTM_Model(b,hyperparameter,'solar_generation')
        lstm_model.save('my_models/models/LSTM/solar_generation_model_b'+str(i)+'.h5')

    if model_type == 'bi-lstm':
        lstm_model = LSTM_Bi_Model_hyper(b,hyperparameter,'solar_generation')
        lstm_model.save('my_models/models/LSTM_Bi_Model/solar_generation_model_b'+str(i)+'_hyper.h5')
    i = i + 1

## Helper Functions

### LightGBM Models

In [82]:
def LightGBM_Model(b,hpt,feature):

    # Load the feature selection
    f_l = pd.read_csv('data/features/feature_importance_cooling_demand.csv')
    
    
    # Generate the x,y
    features = b#[f_l['feature']]
    target   = b[feature]
    print(str(target))
    
    # Generate the test,train 
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42, shuffle=False)
    
    
    
    if hpt == True:
        params = {
            'max_depth':        [3, 4, 5],
            'num_leaves':       [10, 15, 20],
            'learning_rate':    [0.05, 0.1, 0.15],
            'n_estimators':     [50, 100, 200],
            'subsample':        [0.5, 0.7, 0.9],
            'colsample_bytree': [0.5, 0.7, 0.9],
            'reg_alpha':        [0.01, 0.1, 1],
            'reg_lambda':       [0.01, 0.1, 1],
            'steps':            [48],
            'verbose':[-1]
        }
    
        lgb_mean = LGBMRegressor(boosting_type='gbdt', objective='regression')
        grid_search_mean = GridSearchCV(lgb_mean, params, cv=5, n_jobs=-1)
        grid_search_mean.fit(X_train, y_train)
        
        # Create an AdaBoost model with LightGBM as the base estimator
        #adaboost_model = AdaBoostRegressor(base_estimator=grid_search_mean, n_estimators=50)
        #adaboost_model.fit(X_train, y_train)
        
        return grid_search_mean
    
    
    else:
        lgb_params = {
        'n_jobs': 1,
        'max_depth': 4,
        'min_data_in_leaf': 10,
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'subsample': 0.9,
        'n_estimators': 80,
        'learning_rate': 0.1,
        'colsample_bytree': 0.9,
        'steps':48,
        'verbose':-1,
        }
                
        # fitting the model
        gbm = LGBMRegressor(**lgb_params)
        gbm.fit(X_train, y_train)
        print(str(len(gbm.booster_.feature_name())))
        # Create an AdaBoost model with LightGBM as the base estimator
        #adaboost_model = AdaBoostRegressor(base_estimator=gbm, n_estimators=50)
        #adaboost_model.fit(X_train, y_train)

        return gbm#adaboost_model

### LightGBM Multistep

In [40]:
def MultiStepLightGBM(b,hpt,feature):
    
    lgb_params = {
        'n_jobs': 1,
        'max_depth': 4,
        'min_data_in_leaf': 10,
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'subsample': 0.9,
        'n_estimators': 80,
        'learning_rate': 0.1,
        'colsample_bytree': 0.9,
        'verbose':-1,
        }
    

    # Prepare lagged features for 48 steps ahead prediction
    n_steps = 48
    for i in range(1, n_steps + 1):
        b[f'lag_{i}'] = b[feature].shift(-i)
        
    
    X = b
    y = b[feature]
    
    # Generate the test,train 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
    
    
    # Standardize the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
        
    # Create LightGBM dataset
    train_data = lgb.Dataset(X_train_scaled, label=y_train)
    test_data = lgb.Dataset(X_test_scaled, label=y_test, reference=train_data)     
        
    
    # Train the LightGBM model
    num_round = 1000
    bst = lgb.train(lgb_params, train_data, num_round, valid_sets=[test_data])
    
    return bst
    

In [ ]:
n_steps = 48
for i in range(1, n_steps + 1):
    print(str(i))

### CatBoost Models

In [41]:
def CatBoost_Model(b,hpt,feature):
    
    # Load the feature selection
    f_l = pd.read_csv('data/features/feature_importance_'+str(feature)+'.csv')
    
    
    # Generate the x,y
    features = b#[f_l['feature']]
    target   = b[feature]

    
    # Generate the test,train 
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42, shuffle=False)
    
    reg = XGBRegressor(n_estimators=100)
    reg.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            early_stopping_rounds=50)

    return reg
    

### LSTM Model

In [42]:
def LSTM_Bi_Model_hyper(b, hyperparameter, feature):
    
    # Generate the x,y
    X = b
    y = b[feature]
    
    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
    # Reshape data for LSTM input (samples, sequence_length, num_features)
    X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))

    def objective(params, X_train, X_val, y_train, y_val):
        units, learning_rate, dropout_input, dropout_lstm, num_layers, batch_size = params

        # Reshape the input data to match the LSTM input shape
        input_shape = (X_train.shape[1], X_train.shape[2])

        model = Sequential()
        model.add(Bidirectional(LSTM(units=units, return_sequences=True, dropout=dropout_input, recurrent_dropout=dropout_lstm), input_shape=input_shape))

        for _ in range(num_layers - 1):
            model.add(Bidirectional(LSTM(units=units, return_sequences=True, dropout=dropout_lstm, recurrent_dropout=dropout_lstm)))

        model.add(LSTM(units=units, dropout=dropout_lstm, recurrent_dropout=dropout_lstm))
        model.add(Dense(1))

        optimizer = Adam(learning_rate=learning_rate)
        model.compile(optimizer=optimizer, loss='mean_squared_error')
        model.fit(X_train, y_train, epochs=100, batch_size=batch_size, validation_data=(X_val, y_val), verbose=0)
        val_loss = model.evaluate(X_val, y_val)
        return val_loss

    space = [
        (50, 150),                           # Number of LSTM units (units in each LSTM layer)
        (1e-6, 1e-2, 'log-uniform'),         # Learning rate
        (0.1, 0.9),                          # Dropout rate for input layer
        (0.1, 0.9),                          # Dropout rate for LSTM layers
        (1, 3),                              # Number of LSTM layers
        (10, 100)                            # Batch size
    ]

    # Create a partial function without fixed arguments
    objective_partial = partial(objective, X_train=X_train, X_val=X_test, y_train=y_train, y_val=y_test)

    # Perform hyperparameter optimization
    result = gp_minimize(objective_partial, space, n_calls=20, random_state=42)

    # Get the best hyperparameters
    best_units, best_learning_rate, best_dropout_input, best_dropout_lstm, best_num_layers, best_batch_size = result.x
    best_val_loss = result.fun

    # Train the final model using the best hyperparameters
    best_model = Sequential()
    best_model.add(Bidirectional(LSTM(units=best_units, return_sequences=True, dropout=best_dropout_input, recurrent_dropout=best_dropout_lstm), input_shape=(X_train.shape[1], X_train.shape[2])))
    
    for _ in range(best_num_layers - 1):
        best_model.add(Bidirectional(LSTM(units=best_units, return_sequences=True, dropout=best_dropout_lstm, recurrent_dropout=best_dropout_lstm)))
    
    best_model.add(LSTM(units=best_units, dropout=best_dropout_lstm, recurrent_dropout=best_dropout_lstm))
    best_model.add(Dense(1))
    
    optimizer = Adam(learning_rate=best_learning_rate)
    best_model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    # Use early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    best_model.fit(X_train, y_train, epochs=1000, batch_size=best_batch_size, validation_data=(X_test, y_test), verbose=0, callbacks=[early_stopping])

    return best_model


In [43]:
from keras.layers import Bidirectional

def LSTM_Bi_Model(b,hpt,feature):
    
    # Generate the x,y
    features = b
    target   = b[feature]

    
    # Generate the test,train 
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42, shuffle=False)

    
    # Reshape data for LSTM input (samples, sequence_length, num_features)
    X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))
    

    def objective_attention(params, X_train, X_val, y_train, y_val):
        units, learning_rate, dropout_input, dropout_lstm, num_layers, batch_size = params

        # Reshape the input data to match the LSTM input shape
        input_shape = (X_train.shape[1], X_train.shape[2])

        # Input layer
        inputs = Input(shape=input_shape)

        # Bidirectional LSTM layers
        lstm_output = Bidirectional(LSTM(units=units, return_sequences=True, dropout=dropout_input, recurrent_dropout=dropout_lstm))(inputs)
        for _ in range(num_layers - 1):
            lstm_output = Bidirectional(LSTM(units=units, return_sequences=True, dropout=dropout_lstm, recurrent_dropout=dropout_lstm))(lstm_output)

        # BahdanauAttention layer
        context_vector, attention_weights = BahdanauAttention(units)(lstm_output, lstm_output)

        # Concatenate context vector with LSTM output
        lstm_output_combined = Concatenate()([context_vector, lstm_output])

        # LSTM layer with return_sequences=False, go_backwards=True
        lstm = LSTM(units=units, return_sequences=False, go_backwards=True)(lstm_output_combined)

        # Output layer
        output = Dense(48)(lstm)

        # Create the model
        model = Model(inputs=inputs, outputs=output)

        optimizer = Adam(learning_rate=learning_rate)
        model.compile(optimizer=optimizer, loss='mean_squared_error')
        model.fit(X_train, y_train, epochs=100, batch_size=batch_size, validation_data=(X_val, y_val), verbose=0)
        val_loss = model.evaluate(X_val, y_val)
        return val_loss    
    
    
    
    space = [
        (50, 150),                           # Number of LSTM units (units in each LSTM layer)
        (1e-6, 1e-2, 'log-uniform'),         # Learning rate
        (0.1, 0.9),                          # Dropout rate for input layer
        (0.1, 0.9),                          # Dropout rate for LSTM layers
        (1, 3),                              # Number of LSTM layers
        (10, 100)                            # Batch size
    ]

    # Create a partial function without fixed arguments
    objective_partial = partial(objective_attention, X_train=X_train, X_val=X_test, y_train=y_train, y_val=y_test)

    # Perform hyperparameter optimization
    result = gp_minimize(objective_partial, space, n_calls=20, random_state=42)

    # Get the best hyperparameters
    best_units, best_learning_rate, best_dropout_input, best_dropout_lstm, best_num_layers, best_batch_size = result.x
    best_val_loss = result.fun

    # Train the final model using the best hyperparameters
    best_model = Sequential()
    best_model.add(Bidirectional(LSTM(units=best_units, return_sequences=True, dropout=best_dropout_input, recurrent_dropout=best_dropout_lstm), input_shape=(X_train.shape[1], X_train.shape[2])))
    
    for _ in range(best_num_layers - 1):
        best_model.add(Bidirectional(LSTM(units=best_units, return_sequences=True, dropout=best_dropout_lstm, recurrent_dropout=best_dropout_lstm)))
    
    best_model.add(LSTM(units=best_units, dropout=best_dropout_lstm, recurrent_dropout=best_dropout_lstm))
    best_model.add(Dense(48))
    
    optimizer = Adam(learning_rate=best_learning_rate)
    best_model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    # Use early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    best_model.fit(X_train, y_train, epochs=1000, batch_size=best_batch_size, validation_data=(X_test, y_test), verbose=0, callbacks=[early_stopping])

    return best_model
    
    # BiDirectional LSTM with Attention:
    #inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
    #bidirectional_lstm = Bidirectional(LSTM(units=50, return_sequences=True))(inputs)
    #bidirectional_lstm2 = Bidirectional(LSTM(units=50, return_sequences=True))(bidirectional_lstm)
    #lstm, forward_h, forward_c, backward_h, backward_c = LSTM(units=50, return_state=True, go_backwards=True)(bidirectional_lstm2)
    #state_h = Concatenate()([forward_h, backward_h])
    #context_vector, attention_weights = BahdanauAttention(50)(lstm, lstm)

    #output = Dense(units=48)(context_vector)
    #model = Model(inputs=inputs, outputs=output)

    #model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Use early stopping to prevent overfitting
    #early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    #model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping],verbose=0)
    #return model

In [44]:
#from keras.layers import Bidirectional

def LSTM_Model(b,hpt,feature):

    # Generate the x,y
    features = b
    target   = b[feature]

    
    # Generate the test,train 
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42, shuffle=False)
    
    # Reshape data for LSTM input (samples, sequence_length, num_features)
    X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))
    
    # LSTM with Attention: 
    inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
    lstm = LSTM(units=50, return_sequences=True)(inputs)
    lstm, forward_h, forward_c = LSTM(units=50, return_state=True, go_backwards=True)(lstm)
    state_h = Concatenate()([forward_h, forward_c])
    context_vector, attention_weights = BahdanauAttention(50)(lstm, lstm)

    output = Dense(units=48)(context_vector)
    model = Model(inputs=inputs, outputs=output)

    model.compile(optimizer='adam', loss='mean_squared_error')
    

    # Use early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping],verbose=0)


    return model

In [45]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input, Concatenate
import tensorflow as tf

class BahdanauAttention(Layer):
    def __init__(self, units, **kwargs):
        super(BahdanauAttention, self).__init__(**kwargs)
        self.units = units
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def build(self, input_shape):
        # Build the custom layer here if necessary
        super(BahdanauAttention, self).build(input_shape)

    def call(self, query, values):
        # Expand dimensions for broadcasting
        query_with_time_axis = tf.expand_dims(query, 1)

        # Calculate attention scores
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))

        # Compute attention weights
        attention_weights = tf.nn.softmax(score, axis=1)

        # Calculate context vector
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

    def get_config(self):
        config = {
            'units': self.units,
        }
        base_config = super(BahdanauAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [46]:
def Fusion_Model(b,hpt,feature,lstm_model,lightgbm_model):
    
    # Load the feature selection
    f_l = pd.read_csv('data/features/feature_importance_'+str(feature)+'.csv')
    
    
    # Generate the x,y
    features = b#[f_l['feature']]
    target   = b[feature]

    
    # Generate the test,train 
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42, shuffle=False)
    
    X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))
    
    # Create an ensemble model using VotingRegressor
    ensemble_model = VotingRegressor(estimators=[('lstm', lstm_model), ('lgbm', lightgbm_model)])
    ensemble_model.fit(X_train, y_train)
    
    return ensemble_model
    

In [ ]:
!pip install pmdarima

In [ ]:
from sktime.forecasting.arima import AutoARIMA
from sktime.datasets import load_longley
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon


y, X = load_longley()
y_train, y_test, X_train, X_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
fh = np.arange(len(X_train)) + 1


forecaster = LGBMRegressor()
forecaster.fit(y_train, X_train)
y_pred = forecaster.predict(X=X_test)

X

In [ ]:
y